In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.datasets import Flowers102
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Preprocess and load the dataset
# Apply data augementation Techniques
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = Flowers102(root='./data', split='train', transform=data_transform, download=True)
val_dataset = Flowers102(root='./data', split='val', transform=data_transform, download=True)
test_dataset = Flowers102(root='./data', split='test', transform=data_transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


# Define the neural network architecture
class MyCNN(nn.Module):
    def __init__(self, num_channels=3, num_out_ch=[8, 16], img_w=100, img_h=100, num_classes=102):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=num_out_ch[0],
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(num_out_ch[0])  # Batch normalization after conv1
        self.conv2 = nn.Conv2d(in_channels=num_out_ch[0], out_channels=num_out_ch[1],
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn2 = nn.BatchNorm2d(num_out_ch[1])  # Batch normalization after conv2
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.fc = nn.Linear(in_features=int(img_w/4)*int(img_h/4)*num_out_ch[1], out_features=num_classes)
        self.dropout = nn.Dropout(0.5)  # Dropout layer with p=0.5

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)  # Apply dropout after pooling
        x = self.fc(x.reshape(x.shape[0], -1))
        return x

# Initialize the model
num_classes = len(set(train_dataset._labels))
model = MyCNN(num_channels=3, num_out_ch=[16, 32], img_w=224, img_h=224, num_classes=num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Train the model
epochs = 100
train_losses = []
val_losses = []
test_losses = []
train_accuracy = []
val_accuracy = []
test_accuracy = []
best_test_loss = float("inf")
patience = 3
bad_counter = 0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    train_accuracy.append(100 * correct_train / total_train)

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    val_loss = val_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracy.append(100 * correct_val / total_val)

    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    test_loss = test_loss / len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracy.append(100 * correct_test / total_test)

    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {(100*correct_val / total_val):.2f}%')

    # Early stopping and learning rate adjustment
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        bad_counter = 0
    else:
        bad_counter += 1
        if bad_counter >= patience:
            print("Adjusting learning rate...")
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.1  # Reduce learning rate by a factor of 10
            bad_counter = 0  # Reset bad_counter

print(f'Final Test Loss: {test_loss}, Final Test Accuracy: {(100 * correct_test / total_test):.2f}%')

# Plot the training and test losses
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
plt.plot(range(1, epochs+1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Losses')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), train_accuracy, label='Train Accuracy')
plt.plot(range(1, epochs+1), val_accuracy, label='Validation Accuracy')
plt.plot(range(1, epochs+1), test_accuracy, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Test Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

100%|██████████| 344862509/344862509 [00:14<00:00, 23332210.50it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 432260.44it/s]


100%|██████████| 14989/14989 [00:00<00:00, 28103899.27it/s]


Epoch [1/100], Train Loss: 4.9431, Validation Loss: 4.4725, Validation Accuracy: 4.71%


KeyboardInterrupt: 